In [ ]:
import yaml
import pandas as pd
import numpy as np
from message_ix.utils import make_df
from message_ix.models import MESSAGE_ITEMS
from collections.abc import Mapping
from itertools import repeat

In [ ]:
with open('tech_data_02.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)
    
years = [*range(700,730,10)]
regions = ['Westeros', 'Westerlands']
modes = ['standard']
emissions = ['CO2','CH4']
time = ['year']
first_active_year = tech_data['model_data'].get('first_active_year')

In [ ]:
years_dict = {yv:[ya for ya in years if ya >= yv] for yv in years if yv >= first_active_year}

vtg = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

act = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

years_vtg_act = pd.DataFrame(
    {
    'year_vtg':[k for k, v in years_dict.items() for i in range(len(v))],
    'year_act':[ya for value in years_dict.values() for ya in value]
    }
)

In [ ]:
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

In [ ]:
# Basic DataFrame
#group1 = ['inv_cost','technical_lifetime','growth_new_capacity_up','initial_new_capacity_up']
#group2 = ['var_cost','fix_cost','emission_factor','capacity_factor']
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            if parameters[par] == ['node_loc', 'technology', 'year_vtg']:
                kwargs = {'year_vtg': sorted(list(set(years_vtg_act['year_vtg'])))}
            elif all(e in ['node_loc', 'technology', 'year_vtg', 'year_act'] for e in parameters[par]):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                ))
            if 'emission' in parameters[par]:
                data[par] = data[par]*len(emissions)
                for e in range(len(emissions)):
                    data[par][e] = data[par][e].assign(emission=emissions[e])
            if 'mode' in parameters[par]:
                data[par] = data[par]*len(modes)
                for m in range(len(modes)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(mode=modes[m])
            if 'time' in parameters[par]:
                data[par] = data[par]*len(time)
                for t in range(len(time)):
                    for e in range(len(data[par])):
                        data[par][e] = data[par][e].assign(time=time[t])
            

data = {k: pd.concat(v).reset_index(drop=True) for k, v in data.items()}

data

In [ ]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    for tech, diffs in tech_data['model_data'].items():
        if tech != 'first_active_year':
            for reg in regions:
                print(par,tech,reg)
                multiplier = []
                for i in range(len(data[par])):
                    multiplier.append(
                        np.prod([diffs.get(par,{}).get('node_loc',{}).get(reg,1), # by region 
                                 ((1+diffs.get(par,{}).get('year_vtg',{}).get('rate',0))
                                  **(data[par]['year_vtg'][i]-first_active_year)), # by year_vtg
                                 ((1+diffs.get(par,{}).get('year_act',{}).get('rate',0))
                                  **(data.get(par,{}).get('year_act',{}) # by year_act
                                     .get(i,data[par]['year_vtg'][i])-data[par]['year_vtg'][i])),
                                 diffs.get(par,{}).get('mode',{}).get(data.get(par,{}).get('mode',{}).get(i),1), # by mode
                                 diffs.get(par,{}).get('emission',{}).get(data.get(par,{}).get('emission',{}).get(i),1), # by emission
                        ])
                    )

                value = data[par]['value']*multiplier
                data_expand[par].append(
                    data[par].assign(node_loc=reg,value=value)
                   )

data_expand

In [ ]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in group2+group1}

In [ ]:
with pd.ExcelWriter('tech_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)